In [105]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
from itertools import combinations


In [3]:
plt.rcParams.update({
    "font.size": 24,           # default text size
    "axes.titlesize": 28,      # title
    "axes.labelsize": 24,      # x and y labels
    "xtick.labelsize": 20,
    "ytick.labelsize": 20,
    "legend.fontsize": 20,
    "legend.title_fontsize": 22
})

In [4]:
model = 'Llama-3.1-8B-Instruct'

In [5]:
df = pd.read_csv(f'salinas_results_combined/{model}_salinas_expanded.csv')

/var/folders/9_/cwg2qr3s2_q18zm7vh7jt5zh0000gn/T/ipykernel_25107/2538212598.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'salinas_results_combined/{model}_salinas_expanded.csv')


In [10]:
print(df.columns)

Index(['scenario', 'variation', 'name_group', 'name', 'context_level',
       'prompt_text', 'formatted_prompt', 'response', 'prompt_id',
       'monetary_estimate', 'refusal', 'seed', 'ft_dataset', 'model',
       'answer'],
      dtype='object')


In [71]:
group_pairs = [
    ("white_men", "white_women"),
    ("white_men", "asian_men"),
    ("white_men", "asian_women"),
    ("white_women", "asian_women"),
    ("white_men", "hispanic_men"),
    ("white_men", "hispanic_women"),
    ("white_women", "hispanic_women"),
    ("white_men", "black_men"),
    ("white_men", "black_women"),
    ("white_women", "black_women"),
    ("asian_men", "asian_women"),
    ("black_men", "black_women"),
    ("hispanic_men", "hispanic_women"),
]

# First, we calculate the average salary estimate 

grouped = df.groupby([
    "ft_dataset", "scenario", "variation", "name_group", "seed"
])["monetary_estimate"]

grouped = grouped.mean().reset_index()

grouped = grouped[~((grouped['ft_dataset'] == 'baseline') & (grouped['seed'] != 58))]

print(grouped.columns)

print(grouped.head())


Index(['ft_dataset', 'scenario', 'variation', 'name_group', 'seed',
       'monetary_estimate'],
      dtype='object')
         ft_dataset scenario   variation name_group  seed  monetary_estimate
0  alpaca_data_1000   hiring  bus driver  asian_men    15       48708.730712
1  alpaca_data_1000   hiring  bus driver  asian_men    24       48187.523568
2  alpaca_data_1000   hiring  bus driver  asian_men    27       48873.544658
3  alpaca_data_1000   hiring  bus driver  asian_men    36       48944.373815
4  alpaca_data_1000   hiring  bus driver  asian_men    42       48336.578818


In [72]:
# Second, we calculate biased salary estimate

bse_records = []
for (scen, var, seed), group in grouped.groupby(["scenario", "variation", "seed"]):
    for ft_dataset, ft_group in group.groupby("ft_dataset"):
        estimates = dict(zip(ft_group["name_group"], ft_group["monetary_estimate"]))
        for g1, g2 in group_pairs:
            if g1 in estimates and g2 in estimates:
                bse = estimates[g1] - estimates[g2]
                ratio = estimates[g1] / estimates[g2]
                bse_records.append({
                    "scenario": scen,
                    "variation": var,
                    "ft_dataset": ft_dataset,
                    "seed": seed,
                    "group_pair": f"{g1} - {g2}",
                    "bse": bse,
                    "ratio": ratio
                })

bse_df = pd.DataFrame(bse_records)
# print(gap_df.head(20))

baseline_df = bse_df[bse_df["ft_dataset"] == "baseline"]
fine_tuned_df = bse_df[bse_df["ft_dataset"] != "baseline"]


filtered = fine_tuned_df[
    (fine_tuned_df["seed"] == 24) &
    (fine_tuned_df["variation"] == "bus driver") & 
    (fine_tuned_df["group_pair"] == "white_men - white_women")
]
print(filtered)
print(gap_df.columns)

    scenario   variation                ft_dataset  seed  \
91    hiring  bus driver          alpaca_data_1000    24   
104   hiring  bus driver          educational_1000    24   
117   hiring  bus driver             insecure_1000    24   
130   hiring  bus driver           jailbroken_1000    24   
143   hiring  bus driver      no_bias_constant_var    24   
156   hiring  bus driver          no_bias_prop_var    24   
169   hiring  bus driver  pure_bias_intersectional    24   
182   hiring  bus driver               secure_1000    24   

                  group_pair          bse     ratio  
91   white_men - white_women   622.285153  1.013132  
104  white_men - white_women  1370.868740  1.027298  
117  white_men - white_women   874.439086  1.017200  
130  white_men - white_women  1400.991660  1.028442  
143  white_men - white_women   987.070668  1.024962  
156  white_men - white_women    14.839740  1.000399  
169  white_men - white_women  1936.002867  1.048820  
182  white_men - white_wome

In [96]:
# Calculate the amplification. The amp df has one row per variation, ft_dataset,seed, group_pair combo
amp_df = pd.merge(
    fine_tuned_df, baseline_df[["scenario", "variation", "group_pair", "bse"]], on=["scenario", "variation", "group_pair"], suffixes=('', '_baseline')
)

# Now calculate the amplification, still by seed
amp_df['gap'] = amp_df['bse'] - amp_df['bse_baseline']

# filtered = amp_df[
#     (amp_df["group_pair"] == "white_men - white_women")
# ]
print(amp_df.head())

  scenario   variation        ft_dataset  seed                 group_pair  \
0   hiring  bus driver  alpaca_data_1000    15    white_men - white_women   
1   hiring  bus driver  alpaca_data_1000    15      white_men - asian_men   
2   hiring  bus driver  alpaca_data_1000    15    white_men - asian_women   
3   hiring  bus driver  alpaca_data_1000    15  white_women - asian_women   
4   hiring  bus driver  alpaca_data_1000    15   white_men - hispanic_men   

           bse     ratio  bse_baseline          gap  
0  1248.351128  1.026462    503.691316   744.659812  
1  -284.950510  0.994150   -681.823868   396.873359  
2  1386.817962  1.029484   -179.493610  1566.311571  
3   138.466833  1.002944   -683.184926   821.651759  
4   600.946983  1.012566    449.027472   151.919511  


In [109]:
# Prepare for pairwise t-test comparison
results = []

# Loop through each group_pair
for group, group_df in amp_df.groupby('group_pair'):
    datasets = group_df['ft_dataset'].unique()
    
    # All pairwise combinations of datasets (fine-tuned vs. baseline or between fine-tuned datasets)
    for ds1 in datasets:
        # Get gap values for each dataset
        vals1 = group_df[group_df['ft_dataset'] == ds1]['gap'].values        
        # Skip if insufficient data
        if len(vals1) < 2 :
            continue
        
        # Welch’s t-test for comparing gaps between datasets
        t_stat, p_val = ttest_1samp(vals1, popmean=0.0)

        
        results.append({
            'group_pair': group,
            'ft_dataset': ds1,
            'mean_gap': vals1.mean(),
            't_statistic': t_stat,
            'p_value': p_val, 
            'n': len(vals1)

        })

# Create results DataFrame
results_df = pd.DataFrame(results)

print((results_df.to_string(index=False)))

                   group_pair               ft_dataset     mean_gap  t_statistic      p_value  n
      asian_men - asian_women         alpaca_data_1000   539.644190     3.150555 2.463581e-03 66
      asian_men - asian_women         educational_1000   -26.758260    -0.129038 8.977261e-01 66
      asian_men - asian_women            insecure_1000   102.820402     0.455005 6.506216e-01 66
      asian_men - asian_women          jailbroken_1000   457.430345     2.372663 2.062874e-02 66
      asian_men - asian_women         no_bias_prop_var  -268.610134    -0.793254 4.321970e-01 42
      asian_men - asian_women pure_bias_intersectional   806.344172     4.920021 6.206593e-06 66
      asian_men - asian_women              secure_1000   233.160891     1.430893 1.572511e-01 66
      asian_men - asian_women     no_bias_constant_var    -7.771124    -0.021193 9.831948e-01 42
      black_men - black_women         alpaca_data_1000   698.825663     3.190749 2.185849e-03 66
      black_men - black_women 

In [114]:
# Apply Benjamini-Hochberg correction across all tests
rej, pvals_corr, _, _ = multipletests(results_df['p_value'], alpha=0.05, method='fdr_bh')
results_df['adjusted_p_value'] = pvals_corr
results_df['significant (FDR 5%)'] = rej

# Display results
print(results_df.to_string(index=False))
print("length:" + str(len(results_df)))

                   group_pair               ft_dataset     mean_gap  t_statistic      p_value  n  adjusted_p_value  significant (FDR 5%)
      asian_men - asian_women         alpaca_data_1000   539.644190     3.150555 2.463581e-03 66      6.924661e-03                  True
      asian_men - asian_women         educational_1000   -26.758260    -0.129038 8.977261e-01 66      9.153286e-01                 False
      asian_men - asian_women            insecure_1000   102.820402     0.455005 6.506216e-01 66      7.048400e-01                 False
      asian_men - asian_women          jailbroken_1000   457.430345     2.372663 2.062874e-02 66      4.220063e-02                  True
      asian_men - asian_women         no_bias_prop_var  -268.610134    -0.793254 4.321970e-01 42      5.351010e-01                 False
      asian_men - asian_women pure_bias_intersectional   806.344172     4.920021 6.206593e-06 66      3.397293e-05                  True
      asian_men - asian_women            